In [1]:
import pandas as pd

# Demand prediction problem
class DemandPrediction:
    N_DEMAND_INDICATORS = 13;
    # Parameters consist of a bias (intercept) for the sum and one weight for
    # each demand indicator.
    N_PARAMETERS = N_DEMAND_INDICATORS + 1;

    # Construct a Demand prediction problem instance.
    # The parameter "dataset_name" specifies which dataset to use. Valid values
    # are "train" or "test".
    def __init__(self,dataset_name):
      #Load the specified dataset
      if(dataset_name == "train"):
        self.__X, self.__y = DemandPrediction.__load_dataset("data/train.csv");
      elif(dataset_name == "test"):
        self.__X, self.__y = DemandPrediction.__load_dataset("data/test.csv");
      else:
        raise Exception("Only permitted arguments for " +
          "DemandPrediction::__init__ are train and test.")

    # Rectangular bounds on the search space.
    # Returns a 2D array b such that b[i][0] is the minimum permissible value
    # of the ith solution component and b[i][1] is the maximum.
    def bounds():
        return [[-100,100] for i in range(DemandPrediction.N_PARAMETERS)]

    # Check whether the function parameters (weights) lie within the
    # problem's feasible region.
    # There should be the correct number of weights for the predictor function.
    # Each weight should lie within the range specified by the bounds.
    def is_valid(self, parameters):
        if(len(parameters) != DemandPrediction.N_PARAMETERS):
          return False
        #All weights lie within the bounds.
        b = DemandPrediction.bounds();
        for i in range(len(b)):
          if(parameters[i] < b[i][0] or parameters[i] > b[i][1] ):
            return False
        return True;

    # Evaluate a set of parameters on the dataset used by the class instance
    # (train/test).
    # @param parameters An containing the bias and weights to be used to
    #   predict demand.
    # @return The mean absolute error of the predictions on the selected
    # dataset.
    def evaluate(self, parameters):
        abs_error = 0.0;
        for (x, y) in zip(self.__X,self.__y):
            #print(list(self.__X.values))
            #print(x)
            #print(y)
            #print('--')
            y_pred = DemandPrediction.__predict(x,parameters);
            abs_error += abs(y-y_pred);
        abs_error /= len(self.__X);
        return abs_error;

    def __load_dataset(filename):
        if "train.csv" in filename:
            df = pd.read_csv(filename)
            df = df.iloc[:, 1:]
        else:
            df = pd.read_csv(filename,header=None)
            # print(df.head())
        y = df.iloc[:,0].values
        X = df.iloc[:,1:].values
        return X, y

    # Predicts demand based on a weighted sum of demand indicators. You may
    # replace this with something more complex, but will likely have to change
    # the form of the parameters array as well.
    def __predict(demand_indicators, parameters):
        prediction = parameters[0];

        for i in range(1, len(demand_indicators)):
            prediction += demand_indicators[i] * parameters[i];

        return prediction;

In [2]:
import numpy as np
#from DemandPrediction_v2 import DemandPrediction

# Define cost function
def cost_function(x):
    training_problem = DemandPrediction("train")
    error = training_problem.evaluate(x)
    return error

def pso(cost_function, bounds, n_particles, n_iterations, c1, c2, w, verbose = True):
    dimensions = len(bounds)
    
    # Initialize particle positions and velocities
    positions = np.array([np.random.uniform(bounds[:, 0], bounds[:, 1], dimensions) for _ in range(n_particles)])
    velocities = np.array([np.random.uniform(-1, 1, dimensions) for _ in range(n_particles)])
    
    # Initialize personal best and global best
    personal_best_positions = np.copy(positions)
    personal_best_costs = np.array([cost_function(p) for p in personal_best_positions])
    global_best_position = personal_best_positions[np.argmin(personal_best_costs)]
    global_best_cost = np.min(personal_best_costs)

    error_list = []
    # PSO main loop
    for t in range(n_iterations):
        for i in range(n_particles):
            # Update particle velocity
            r1, r2 = np.random.rand(2)
            velocities[i] = w * velocities[i] + c1 * r1 * (personal_best_positions[i] - positions[i]) + c2 * r2 * (global_best_position - positions[i])

            # Update particle position
            positions[i] += velocities[i]

            # Update personal best
            current_cost = cost_function(positions[i])
            if current_cost < personal_best_costs[i]:
                personal_best_positions[i] = positions[i]
                personal_best_costs[i] = current_cost

                # Update global best
                if current_cost < global_best_cost:
                    global_best_position = positions[i]
                    global_best_cost = current_cost

        # Optional: print progress
        if verbose:
            print("Iteration {}/{}, Global Best Cost: {}".format(t + 1, n_iterations, global_best_cost))
        error_list.append(global_best_cost)

    return global_best_cost, global_best_position, error_list




In [6]:
import numpy as np
import multiprocessing as mp
from functools import partial
#from DemandPrediction_v2 import DemandPrediction
#from Main_PSO_v4 import pso

# Define cost function
def cost_function(x):
    training_problem = DemandPrediction("train")
    error = training_problem.evaluate(x)
    return error

def pso_with_hyperparameters(hyperparams):
    c1, c2, w = hyperparams
    cost, pos, err = pso(cost_function, bounds, n_particles, n_iterations, c1, c2, w, False)
    return (cost, pos, hyperparams)

bounds = np.array(DemandPrediction.bounds())

# Set ranges for hyperparameter tuning
c1_range = [0.3, 0.5, 0.7]
c2_range = [0.3, 0.5, 0.7]
w_range = [0.5, 0.7, 0.9]

n_particles = 100
#n_iterations = 1000
n_iterations = 1

# Prepare combinations of hyperparameters
hyperparameter_combinations = [(c1, c2, w) for c1 in c1_range for c2 in c2_range for w in w_range]

def main():
    # Run the multiprocessing pool
    #with mp.Pool(mp.cpu_count()) as pool:
    #    results = pool.map(pso_with_hyperparameters, hyperparameter_combinations)
    results=[]
    for param in hyperparameter_combinations:
        results.append(pso_with_hyperparameters(param))

    # Find the best solution and associated hyperparameters
    best_cost = float('inf')
    best_pos = None
    best_hyperparameters = None

    for cost, pos, hyperparams in results:
        print("c1: {}, c2: {}, w: {}, cost: {}".format(hyperparams[0], hyperparams[1], hyperparams[2], cost))
        if cost < best_cost:
            best_cost = cost
            best_pos = pos
            best_hyperparameters = hyperparams

    print("\nBest hyperparameters found:")
    print("c1: {}, c2: {}, w: {}".format(best_hyperparameters[0], best_hyperparameters[1], best_hyperparameters[2]))
    print("Best training error: {}".format(best_cost))

    # Evaluate the best solution on the test dataset
    test_problem = DemandPrediction("test")
    test_error = test_problem.evaluate(best_pos)
    print("Test error of best solution found while training: {}".format(test_error))

In [7]:
if __name__ == '__main__':
    #mp.freeze_support()
    main()

c1: 0.3, c2: 0.3, w: 0.5, cost: 683.6942169952753
c1: 0.3, c2: 0.3, w: 0.7, cost: 815.6849439700326
c1: 0.3, c2: 0.3, w: 0.9, cost: 841.5000060247515
c1: 0.3, c2: 0.5, w: 0.5, cost: 614.696459971727
c1: 0.3, c2: 0.5, w: 0.7, cost: 384.8393405255209
c1: 0.3, c2: 0.5, w: 0.9, cost: 466.7628845946277
c1: 0.3, c2: 0.7, w: 0.5, cost: 544.0332270369148
c1: 0.3, c2: 0.7, w: 0.7, cost: 649.0164946998162
c1: 0.3, c2: 0.7, w: 0.9, cost: 603.6626700148686
c1: 0.5, c2: 0.3, w: 0.5, cost: 711.1626791381899
c1: 0.5, c2: 0.3, w: 0.7, cost: 748.0304963204728
c1: 0.5, c2: 0.3, w: 0.9, cost: 741.1701796036461
c1: 0.5, c2: 0.5, w: 0.5, cost: 678.6497240390072
c1: 0.5, c2: 0.5, w: 0.7, cost: 458.66046998258605
c1: 0.5, c2: 0.5, w: 0.9, cost: 556.8421173730533
c1: 0.5, c2: 0.7, w: 0.5, cost: 559.1454943811757
c1: 0.5, c2: 0.7, w: 0.7, cost: 442.193342330039
c1: 0.5, c2: 0.7, w: 0.9, cost: 492.7931932791162
c1: 0.7, c2: 0.3, w: 0.5, cost: 620.922305815348
c1: 0.7, c2: 0.3, w: 0.7, cost: 437.3236718639804
c1